# ASL Recognition - Jack Brummer

Hello! My name is Jack Brummer and I am a recent graduate of Stevens Institute of Technology where I received my masters and bachelors in computer science

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_data = pd.read_csv('E:/competitions/asl-fingerspelling/asl-fingerspelling/train.csv')
train_data.head()
train_data.phrase.str.len().max()

31

In [3]:
file_id = 5414471
path = f'E:/competitions/asl-fingerspelling/asl-fingerspelling/train_landmarks/{file_id}.parquet'
sign = pd.read_parquet(path)
sign

,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
sequence_id,,,,,,,,,,,,,,,,,,,,,
1816796431,0,0.710588,0.699951,0.705657,0.691768,0.699669,0.701980,0.709724,0.610405,0.712660,...,-0.245855,-0.269148,-0.129743,-0.251501,-0.278687,-0.266530,-0.152852,-0.257519,-0.275822,-0.266876
1816796431,1,0.709525,0.697582,0.703713,0.691016,0.697576,0.700467,0.709796,0.616540,0.713729,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1816796431,2,0.711059,0.700858,0.706272,0.693285,0.700825,0.703319,0.711549,0.615606,0.715143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1816796431,3,0.712799,0.702518,0.707840,0.694899,0.702445,0.704794,0.712483,0.625044,0.715677,...,-0.370770,-0.408097,-0.185217,-0.325494,-0.343373,-0.328294,-0.203126,-0.315719,-0.326104,-0.314282
1816796431,4,0.712349,0.705451,0.709918,0.696006,0.705180,0.706928,0.712685,0.614356,0.714875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848182207,296,0.657136,0.635888,0.643259,0.619031,0.633084,0.631827,0.630708,0.533120,0.626672,...,-0.143147,-0.139659,-0.066276,-0.130910,-0.127341,-0.106674,-0.083439,-0.124994,-0.119394,-0.101404
1848182207,297,0.655706,0.635570,0.642730,0.618637,0.632830,0.631554,0.630344,0.531868,0.626445,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1848182207,298,0.653681,0.636057,0.643054,0.618643,0.633258,0.631800,0.630059,0.531178,0.625990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Model 

In [5]:
from keras.layers import Input, LSTM, Bidirectional, Concatenate
from keras.models import Sequential, Model

using_drive = True

class ASL_Transformer_Model:
    
    def __init__(self):
        self.train_path = '/kaggle/input/asl-fingerspelling/train.csv' if not using_drive else "E:/competitions/asl-fingerspelling/asl-fingerspelling/train.csv"
        self.train = pd.read_csv(self.train_path)
        self.sequence_ids = self.train['sequence_ids']
        self.num_sequences = len(self.sequence_ids)
        self.max_sequence_length = self.train.phrase.str.len().max()
        self.num_seqs = len(self.train.phrase)
        self.num_decoder_tokens = 10
        self.num_encoder_tokens = 10

        # input texts -> map from sequence_id to train landmark
        self.input_texts = self.train['sequence_id']
        # target texts -> the respective phrase for each sequence id
        self.target_texts = self.train['phrase']

        
    def get_sign_from_path(self, path, sequence_id):
        
        f'/kaggle/input/asl-fingerspelling/train_landmarks/{file_id}.parquet'
        sign = pd.read_parquet(path)
        return sign
    
    def get_frames(self, train_landmark, sequence_id):
        frames = self.train.loc[self.train['sequence_id'] == sequence_id]
        return frames

    def get_phrase(self, sequence_id):
        phrase = self.train.loc[self.train['sequence_id'] == sequence_id]['phrase']
        return phrase

    def encoder(self):
        latent_dim = 256

        # inputs of the encoder network
        encoder_inputs = Input(shape=(None, self.num_encoder_tokens), 
                            name='encoder_inputs')

        # set the LSTM layer
        encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
                            dropout=0.5, name='encoder_lstm'))

        _, forward_h, forward_c, back_h, back_c = encoder_bilstm(encoder_inputs)

        # build the encoder network model

        state_h = Concatenate()([forward_h, back_h])
        state_c = Concatenate()([forward_c, back_c])

        encoder_model = Model(inputs=encoder_inputs, 
                            outputs=[state_h, state_c],
                            name='encoder')
        return encoder_model
    
    def decoder(self):
        decoder_input_h = Input(shape=(512,), name='decoder_input_h')
        decoder_input_c = Input(shape=(512,), name='decoder_input_c')
        decoder_input_x = Input(shape=(None, self.num_decoder_tokens), name='decoder_input_x')

        # set the LSTM layer
        decoder_lstm = LSTM(512, return_sequences=True, 
                            return_state=True, dropout=0.5, name='decoder_lstm')
        decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x, 
                                                            initial_state=[decoder_input_h, decoder_input_c]) # type: ignore

        # set the dense layer
        decoder_dense = Dense(self.num_decoder_tokens, activation='softmax', name='decoder_dense')
        decoder_outputs = decoder_dense(decoder_lstm_outputs)

        # build the decoder network model
        decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                            outputs=[decoder_outputs, state_h, state_c],
                            name='decoder')
        
        return decoder_model
    
    

In [ ]:
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences

def makeSequence(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = makeSequence(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = makeSequence(max_decoder_seq_length, 
                                                       target_texts)